In [87]:
'''
Challenge link
https://preppindata.blogspot.com/2023/03/2023-week-9-customer-bank-statements.html
'''
import pandas as pd
import numpy as np

In [88]:
acc_info = pd.read_csv('Account Information.csv')
trans_details = pd.read_csv("Transaction Detail.csv")
trans_path = pd.read_csv("Transaction Path.csv")

acc_info.head(2) , trans_details.head(2) , trans_path.head(2)

(   Account Number Account Type Account Holder ID Balance Date  Balance
 0        10005367     Platinum          70390615   2023-01-31   728.25
 1        10011977        Basic          20123998   2023-01-31   676.54,
    Transaction ID Transaction Date   Value Cancelled?
 0         1957155       2023-02-01  128.78          N
 1        28234510       2023-02-01  163.82          N,
    Transaction ID  Account_To  Account_From
 0         1957155    27356852      76206810
 1         2147025    44242297      24826358)

### For the Transaction Path table:
        Make sure field naming convention matches the other tables
        i.e. instead of Account_From it should be Account From

In [89]:
trans_path.rename(columns={'Account_To' : 'Account To', 'Account_From' : 'Account From'}, inplace=True)
trans_path.head(2)

,Transaction ID,Account To,Account From
0,1957155,27356852,76206810
1,2147025,44242297,24826358


### Filter out the cancelled transactions

In [90]:
trans_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8781 entries, 0 to 8780
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    8781 non-null   int64  
 1   Transaction Date  8781 non-null   object 
 2   Value             8781 non-null   float64
 3   Cancelled?        8781 non-null   object 
dtypes: float64(1), int64(1), object(2)
memory usage: 274.5+ KB


In [91]:
trans_details = trans_details[trans_details['Cancelled?'] == 'N']
trans_details = trans_details.reset_index()
trans_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8689 entries, 0 to 8688
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   index             8689 non-null   int64  
 1   Transaction ID    8689 non-null   int64  
 2   Transaction Date  8689 non-null   object 
 3   Value             8689 non-null   float64
 4   Cancelled?        8689 non-null   object 
dtypes: float64(1), int64(2), object(2)
memory usage: 339.5+ KB


### Split the flow into incoming and outgoing transactions

In [92]:
trans_path.info() , trans_path.head(2)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8781 entries, 0 to 8780
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   Transaction ID  8781 non-null   int64
 1   Account To      8781 non-null   int64
 2   Account From    8781 non-null   int64
dtypes: int64(3)
memory usage: 205.9 KB


(None,
    Transaction ID  Account To  Account From
 0         1957155    27356852      76206810
 1         2147025    44242297      24826358)

In [93]:
temp = trans_details.merge(trans_path, how='left', on='Transaction ID')
temp = temp.reset_index()
temp.head(5)

,level_0,index,Transaction ID,Transaction Date,Value,Cancelled?,Account To,Account From
0,0,0,1957155,2023-02-01,128.78,N,27356852,76206810
1,1,1,28234510,2023-02-01,163.82,N,28745450,87373821
2,2,2,33688648,2023-02-01,54.71,N,48271608,98821521
3,3,3,41670299,2023-02-01,88.10,N,34128127,80808205
4,4,5,57723869,2023-02-01,89.22,N,12859818,34902863


In [94]:
temp.drop(columns=['level_0', 'index' , 'Cancelled?'], inplace=True)
temp.head(2)

,Transaction ID,Transaction Date,Value,Account To,Account From
0,1957155,2023-02-01,128.78,27356852,76206810
1,28234510,2023-02-01,163.82,28745450,87373821


In [95]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8689 entries, 0 to 8688
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    8689 non-null   int64  
 1   Transaction Date  8689 non-null   object 
 2   Value             8689 non-null   float64
 3   Account To        8689 non-null   int64  
 4   Account From      8689 non-null   int64  
dtypes: float64(1), int64(3), object(1)
memory usage: 339.5+ KB


In [96]:
trans_df = pd.DataFrame(columns=['Transaction ID', 'Transaction Date', 'Value', 'Account Number' , 'Flow'])
for i in range(len(temp)):
    row = temp.loc[i].tolist()
    # outgoing
    l = len(trans_df)
    out = row[:4]
    out.append('Outgoing')
    trans_df.loc[l] = out

    # incoming
    l = len(trans_df)
    income = row[:3]
    income.append(row[-1])
    income.append('Incoming')
    trans_df.loc[l] = income

In [97]:
trans_df.head(5)

,Transaction ID,Transaction Date,Value,Account Number,Flow
0,1957155,2023-02-01,128.78,27356852,Outgoing
1,1957155,2023-02-01,128.78,76206810,Incoming
2,28234510,2023-02-01,163.82,28745450,Outgoing
3,28234510,2023-02-01,163.82,87373821,Incoming
4,33688648,2023-02-01,54.71,48271608,Outgoing


In [98]:
trans_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17378 entries, 0 to 17377
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Transaction ID    17378 non-null  int64  
 1   Transaction Date  17378 non-null  object 
 2   Value             17378 non-null  float64
 3   Account Number    17378 non-null  int64  
 4   Flow              17378 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 814.6+ KB


### Bring the data together with the Balance as of 31st Jan

In [99]:
acc_info.head(5)

,Account Number,Account Type,Account Holder ID,Balance Date,Balance
0,10005367,Platinum,70390615,2023-01-31,728.25
1,10011977,Basic,20123998,2023-01-31,676.54
2,10024680,Platinum,54374080,2023-01-31,567.46
3,10031238,Basic,97027297,2023-01-31,576.52
4,10034341,Joint,"89920386, 97325900",2023-01-31,390.39


In [100]:
temp = acc_info
temp.drop(columns=['Account Type' , 'Account Holder ID'], inplace=True)
temp['Transaction Value'] = np.nan
temp['Flow'] = '-'
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Account Number     3000 non-null   int64  
 1   Balance Date       3000 non-null   object 
 2   Balance            3000 non-null   float64
 3   Transaction Value  0 non-null      float64
 4   Flow               3000 non-null   object 
dtypes: float64(2), int64(1), object(2)
memory usage: 117.3+ KB


In [86]:
trans_df.head(5)

,Transaction ID,Transaction Date,Value,Account Number,Flow
0,1957155,2023-02-01,128.78,27356852,Outgoing
1,1957155,2023-02-01,128.78,76206810,Incoming
2,28234510,2023-02-01,163.82,28745450,Outgoing
3,28234510,2023-02-01,163.82,87373821,Incoming
4,33688648,2023-02-01,54.71,48271608,Outgoing


In [101]:
total_df = temp
for i in range(len(trans_df)):
    x = trans_df.loc[i]
    total_df.loc[len(total_df)] = [x[3], x[1], 0 , x[2], x[4]]

total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20378 entries, 0 to 20377
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Account Number     20378 non-null  int64  
 1   Balance Date       20378 non-null  object 
 2   Balance            20378 non-null  float64
 3   Transaction Value  17378 non-null  float64
 4   Flow               20378 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 955.2+ KB


### Work out the order that transactions occur for each account
        Hint: where multiple transactions happen on the same day, assume the highest value transactions happen first

In [106]:
total_df['Balance Date'] = total_df['Balance Date'].astype('datetime64[ns]')
total_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20378 entries, 0 to 20377
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Account Number     20378 non-null  int64         
 1   Balance Date       20378 non-null  datetime64[ns]
 2   Balance            20378 non-null  float64       
 3   Transaction Value  17378 non-null  float64       
 4   Flow               20378 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 955.2+ KB


In [107]:
total_df.head(8)

,Account Number,Balance Date,Balance,Transaction Value,Flow
0,10005367,2023-01-31,728.25,NaN,-
1,10011977,2023-01-31,676.54,NaN,-
2,10024680,2023-01-31,567.46,NaN,-
3,10031238,2023-01-31,576.52,NaN,-
4,10034341,2023-01-31,390.39,NaN,-
5,10079965,2023-01-31,799.27,NaN,-
6,10091076,2023-01-31,831.83,NaN,-
7,10231977,2023-01-31,61.63,NaN,-


In [119]:

temp = total_df.sort_values(['Account Number'], ascending=True).groupby(['Account Number'], sort=False).apply(lambda x: x.sort_values(['Balance Date'], ascending=True)).reset_index(drop=True)
temp.tail(8)

,Account Number,Balance Date,Balance,Transaction Value,Flow
20370,99937043,2023-02-02,0.0,142.45,Incoming
20371,99937043,2023-02-03,0.0,52.49,Incoming
20372,99937043,2023-02-04,0.0,123.12,Outgoing
20373,99937043,2023-02-05,0.0,52.36,Outgoing
20374,99937043,2023-02-06,0.0,92.42,Incoming
20375,99937043,2023-02-09,0.0,122.91,Incoming
20376,99937043,2023-02-11,0.0,125.33,Incoming
20377,99937043,2023-02-13,0.0,75.16,Outgoing


In [121]:
temp.head(5)

,Account Number,Balance Date,Balance,Transaction Value,Flow
0,10005367,2023-01-31,728.25,NaN,-
1,10005367,2023-02-02,0.00,1097.60,Incoming
2,10005367,2023-02-04,0.00,144.75,Outgoing
3,10005367,2023-02-04,0.00,67.01,Incoming
4,10005367,2023-02-05,0.00,108.26,Incoming


### Use a running sum to calculate the Balance for each account on each day

In [125]:
id = 0
bal = 0
for i in range(len(temp)):
    # same Account Number as previous i
    if id == temp.loc[i,'Account Number']:
        # add to balance
        if temp.loc[i,'Flow'] == 'Incoming':
            temp.loc[i,'Balance'] = bal + temp.loc[i,'Transaction Value']
        elif temp.loc[i,'Flow'] == 'Outgoing':
            temp.loc[i,'Balance'] = bal - temp.loc[i,'Transaction Value']
        bal = temp.loc[i,'Balance']
    # new Account Number
    else:
        id = temp.loc[i,'Account Number']
        bal = temp.loc[i,'Balance']
        

In [126]:
temp.head(12)

,Account Number,Balance Date,Balance,Transaction Value,Flow
0,10005367,2023-01-31,728.25,NaN,-
1,10005367,2023-02-02,1825.85,1097.60,Incoming
2,10005367,2023-02-04,1681.10,144.75,Outgoing
3,10005367,2023-02-04,1748.11,67.01,Incoming
4,10005367,2023-02-05,1856.37,108.26,Incoming
5,10005367,2023-02-08,1921.86,65.49,Incoming
6,10005367,2023-02-08,980.86,941.00,Outgoing
7,10005367,2023-02-10,1008.47,27.61,Incoming
8,10005367,2023-02-11,871.22,137.25,Outgoing
9,10011977,2023-01-31,676.54,NaN,-


In [127]:
temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20378 entries, 0 to 20377
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Account Number     20378 non-null  int64         
 1   Balance Date       20378 non-null  datetime64[ns]
 2   Balance            20378 non-null  float64       
 3   Transaction Value  17378 non-null  float64       
 4   Flow               20378 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 796.1+ KB


In [ ]:
temp.to_csv('output.csv' , index=False)